# Stage 5: Speed of the CME

1.   Calculate the speed of the CME using observations.

Pick a coronagraph image where the CME bubble is clearly visible by selecting a frame nubmer in the box below. If you are happy with the image move on to the next cell. If not, run this cell again and pick another number.

In [ ]:
!pip install plotly

In [ ]:
import glob
import cv2
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from shutil import rmtree
from os import makedirs as newdir
from os import path


if path.exists('WEW_PythonWorkshop_Database/'): # You wouldn't need these two lines if you're plotting a different date!
    rmtree(path.join('WEW_PythonWorkshop_Database/'))

! git clone https://github.com/andyto1234/WEW_PythonWorkshop_Database

#Fetch the coronagraph images
map_list = []
map_list = sorted(glob.glob('WEW_PythonWorkshop_Database/LASCO_images/*.png'))

filenumber = input('Image number: ')
filename = map_list[int(filenumber)]

#Read and plot the image file
img = cv2.imread(filename)
fig=px.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), width=1000, height=600)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.show()

Now select two points on the image to calculate the distance the CME has travelled. First point should be at the center or surface of the Sun. Second point should be at the top of the CME bubble.


In [ ]:
coords= []
x1 = input('CME source x coordinate: ')
y1 = input('CME source y coordinate: ')
coords.append([int(x1), int(y1)])

x2 = input('CME front x coordinate: ')
y2 = input('CME front y coordinate: ')
coords.append([int(x2), int(y2)])

dist = np.linalg.norm(np.array(coords[1])-np.array(coords[0]))*63303
print('CME has travelled ', int(dist), ' km')

Enter the time the CME is observed at its origin on the Sun and in the coronagraph.

In [ ]:
t1 = input('Time plasma structure erupted on the Sun (hh:mm)')
t2 = input('Time CME is spotted in the coronagraph (hh:mm)')

t1_hour, t1_minute = [i for i in t1.split(":")]
t1 = int(t1_hour)*60*60 + int(t1_minute)*60

t2_hour, t2_minute = [i for i in t2.split(":")]
t2 = int(t2_hour)*60*60 + int(t2_minute)*60

Now calculate the speed.

In [ ]:
v = dist/(t2-t1)
print('CME speed is: ', int(v), ' km/s')

## How soon does it hit the Earth?

When predicting space weather, it is essential to know the speed of a CME so that we can calculate how soon it would reach the Earth. Using the CME speed calculated becofe and knowing the distance between the Earth and the Sun, work out how much time it would take the CME to reach us.
